In [1]:
# connect to google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Imports
import pandas as pd
import numpy as np
import librosa
import os
import json
from joblib import Parallel, delayed
import math

In [3]:
# Get data into colab
ZIP_PATH = "/content/drive/My Drive/Summer 2025/ESC-50 Water Project/audio/ESC-50-master.zip"
DESTINATION_DIR = "/content/"

!unzip -q "{ZIP_PATH}" -d "{DESTINATION_DIR}"

In [4]:
# Create constants

# File paths
ESC50_DIR = "/content/ESC-50-master/"
AUDIO_DIR = os.path.join(ESC50_DIR, "audio")
CSV_PATH = os.path.join(ESC50_DIR, "meta/esc50.csv")

# Directory to save the new .npz files
OUTPUT_DIR_NPZ = "/content/drive/My Drive/Summer 2025/ESC-50 Water Project/audio/npz_files"

# Audio processing constant
SAMPLE_RATE = 22050

In [5]:
# Load csv using pandas
df = pd.read_csv(CSV_PATH)

'''
Will only be focusing on water and non-water categories making this a binary
classification problem
'''

# categories to be considered as water
water_categories = ['pouring_water', 'toilet_flush', 'water_drops']

# Make a new column called is_water

# .astype(int) changes the True or False val to 1 or 0
df['is_water'] = df['category'].isin(water_categories).astype(int)

# Print the first 5 rows for verification
print("DataFrame with new 'is_water' column:")
print(df.head())

# Print counts of each class to see the totals
print("Totals:")
print(df['is_water'].value_counts())

DataFrame with new 'is_water' column:
            filename  fold  target        category  esc10  src_file take  \
0   1-100032-A-0.wav     1       0             dog   True    100032    A   
1  1-100038-A-14.wav     1      14  chirping_birds  False    100038    A   
2  1-100210-A-36.wav     1      36  vacuum_cleaner  False    100210    A   
3  1-100210-B-36.wav     1      36  vacuum_cleaner  False    100210    B   
4  1-101296-A-19.wav     1      19    thunderstorm  False    101296    A   

   is_water  
0         0  
1         0  
2         0  
3         0  
4         0  
Totals:
is_water
0    1880
1     120
Name: count, dtype: int64


In [6]:
# Separate the two categories
df_water = df[df['is_water'] == 1]
df_no_water = df[df['is_water'] == 0]

'''
Randomly sample the 'no_water' class to match the number of 'water' samples.
This ensures a balanced dataset for training allowing the model to learn
patterns for the classification of water.
'''
df_no_water_balanced = df_no_water.sample(n=len(df_water), random_state=42)

# Combine the balanced dataframes
df_balanced = pd.concat([df_water, df_no_water_balanced])

# Print counts of each class to see the totals
print("Balanced DataFrame Totals:")
print(df_balanced['is_water'].value_counts())

Balanced DataFrame Totals:
is_water
1    120
0    120
Name: count, dtype: int64


In [ ]:
def process_audio_file(file_path, label, num_mfcc, n_fft, hop_length, segment_duration_s):
    """
    Loads one 5-second audio file, extracts its MFCCs, and slices them
    into smaller segments for a certain duration

    Args:
        file_path (str): Path to the audio file
        label (int): Label for the audio file
        num_mfcc (int): Number of MFCCs to extract
        n_fft (int): FFT size
        hop_length (int): Hop length for the FFT
        segment_duration_s (float): Duration of each segment in seconds

    Returns:
        mfcc_segments (list): List of MFCC segments
        labels_for_segments (list): List of labels for each segment
    """

    try:
        # Load audio file
        signal, sr = librosa.load(file_path, sr=SAMPLE_RATE)

        # Extract MFCCs for the whole audio
        mfcc_full = librosa.feature.mfcc(y=signal, sr=sr, n_mfcc=num_mfcc, n_fft=n_fft, hop_length=hop_length)

        # For segmentation
        mfcc_segments = []
        labels_for_segments = []

        total_frames = mfcc_full.shape[1]
        samples_per_segment = int(segment_duration_s * sr)

        # Calculate how many MFCC frames are in each segment
        frames_per_segment = math.ceil(samples_per_segment / hop_length)

        # Calculate the number of segments that fit in the clip
        num_segments = total_frames // frames_per_segment

        # Snip the mfcc into it's segments
        for i in range(num_segments):
            start = i * frames_per_segment
            end = start + frames_per_segment
            # Get the segment and transpose so time is the first dimension
            segment = mfcc_full[:, start:end].T
            mfcc_segments.append(segment)
            labels_for_segments.append(label)

        return mfcc_segments, labels_for_segments
    # got a bad audio file
    except Exception as e:
        print(f"Error processing {file_path}: {e}")
        return [], []

In [ ]:
def save_features_to_npz(dataframe, num_mfcc=32, n_fft=512, overlap=0.5, segment_duration_s=1.0):
    """
    Manages the extraction of MFCCs from all audio files and saves to a .npz file.
    Uses parallel processing to speed up the process.

    Args:
        dataframe (pd.DataFrame): DataFrame containing audio file paths and labels
        num_mfcc (int): Number of MFCCs to extract
        n_fft (int): FFT size
        overlap (float): Percentage of overlap between segments
        segment_duration_s (float): Duration of each segment in seconds
    """

    # Calculate hop_length using n_fft and the overlap percentage
    hop_length = int(n_fft * (1 - overlap))
    overlap_percent = int(overlap * 100)

    # Calculate the duration of each segment in milliseconds for the filename
    duration_ms = int(segment_duration_s * 1000)

    # Create a filename based on the parameters
    npz_filename = f"esc50_mfcc_n{num_mfcc}_fft{n_fft}_ovlp{overlap_percent}_seg{duration_ms}ms.npz"
    output_path = os.path.join(OUTPUT_DIR_NPZ, npz_filename)
    print(f"Starting job for: {npz_filename}")

    # Prepare "Jobs" for paralell processing
    jobs = []
    for index, row in dataframe.iterrows():
        file_path = os.path.join(AUDIO_DIR, row['filename'])
        label = row['is_water']
        # passed as a tuple
        jobs.append((file_path, label))

    # Run Parallel Processing
    results = Parallel(n_jobs=-1, verbose=1)(
        delayed(process_audio_file)(
            file_path, label, num_mfcc, n_fft, hop_length, segment_duration_s
        ) for file_path, label in jobs
    )

    # Collect Results
    # Use .extend() since jobs return a list of segments
    final_mfccs = []
    final_labels = []
    for mfcc_list, label_list in results:
        final_mfccs.extend(mfcc_list)
        final_labels.extend(label_list)

    # Save to .npz
    np.savez_compressed(output_path,
                      mfcc=final_mfccs,
                      labels=final_labels,
                      mapping=['no_water', 'water'])

    print(f"Successfully saved data to: {output_path}\\n")

In [ ]:
#   --- Create MFCCs with different configurations (50% overlap) ---
# Define the parameter ranges to test
segment_durations = [5.0, 2.0, 1.0, 0.5, 0.2, 0.1, 0.05, 0.025]
n_mfccs= [32, 64, 128]
n_fft = [512, 1024, 2048]

# Loop through each combination of parameters
for seg_duration in segment_durations:
  for mfcc in n_mfccs:
    for fft in n_fft:
        save_features_to_npz(df_balanced,
                            num_mfcc=mfcc,
                            n_fft=fft,
                            overlap=0.5,
                            segment_duration_s=seg_duration)

print("--- All preprocessing jobs are complete! ---")

Starting job for: esc50_mfcc_n32_fft512_ovlp50_seg25ms.npz


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done 160 tasks      | elapsed:    6.8s
[Parallel(n_jobs=-1)]: Done 240 out of 240 | elapsed:    7.6s finished


Successfully saved data to: /content/drive/My Drive/Summer 2025/Python Practice/audio/npz_files/esc50_mfcc_n32_fft512_ovlp50_seg25ms.npz\n
Starting job for: esc50_mfcc_n32_fft1024_ovlp50_seg25ms.npz


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done 237 out of 240 | elapsed:    2.1s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done 240 out of 240 | elapsed:    2.2s finished


Successfully saved data to: /content/drive/My Drive/Summer 2025/Python Practice/audio/npz_files/esc50_mfcc_n32_fft1024_ovlp50_seg25ms.npz\n
Starting job for: esc50_mfcc_n32_fft2048_ovlp50_seg25ms.npz


...

Successfully saved data to: /content/drive/My Drive/Summer 2025/Python Practice/audio/npz_files/esc50_mfcc_n128_fft512_ovlp50_seg5000ms.npz\n
Starting job for: esc50_mfcc_n128_fft1024_ovlp50_seg5000ms.npz


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done 240 out of 240 | elapsed:    1.8s finished


Successfully saved data to: /content/drive/My Drive/Summer 2025/Python Practice/audio/npz_files/esc50_mfcc_n128_fft1024_ovlp50_seg5000ms.npz\n
Starting job for: esc50_mfcc_n128_fft2048_ovlp50_seg5000ms.npz


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done 240 out of 240 | elapsed:    1.7s finished


Successfully saved data to: /content/drive/My Drive/Summer 2025/Python Practice/audio/npz_files/esc50_mfcc_n128_fft2048_ovlp50_seg5000ms.npz\n
--- All preprocessing jobs are complete! ---


In [ ]:
#   --- Create MFCCs with different configurations (25% overlap) ---
# Define the parameter ranges to test
segment_durations = [5.0, 2.0, 1.0, 0.5, 0.2, 0.1, 0.05, 0.025]
n_mfccs= [32, 64, 128]
n_fft = [512, 1024, 2048]

# Loop through each combination of parameters
for seg_duration in segment_durations:
  for mfcc in n_mfccs:
    for fft in n_fft:
        save_features_to_npz(df_balanced,
                            num_mfcc=mfcc,
                            n_fft=fft,
                            overlap=0.25,
                            segment_duration_s=seg_duration)

print("--- All preprocessing jobs are complete! ---")

Starting job for: esc50_mfcc_n32_fft512_ovlp25_seg25ms.npz


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done 240 out of 240 | elapsed:    2.0s finished


Successfully saved data to: /content/drive/My Drive/Summer 2025/Python Practice/audio/npz_files/esc50_mfcc_n32_fft512_ovlp25_seg25ms.npz\n
Starting job for: esc50_mfcc_n32_fft1024_ovlp25_seg25ms.npz


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done 240 out of 240 | elapsed:    2.0s finished


Successfully saved data to: /content/drive/My Drive/Summer 2025/Python Practice/audio/npz_files/esc50_mfcc_n32_fft1024_ovlp25_seg25ms.npz\n
Starting job for: esc50_mfcc_n32_fft2048_ovlp25_seg25ms.npz


...

Successfully saved data to: /content/drive/My Drive/Summer 2025/Python Practice/audio/npz_files/esc50_mfcc_n128_fft512_ovlp25_seg5000ms.npz\n
Starting job for: esc50_mfcc_n128_fft1024_ovlp25_seg5000ms.npz


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done 240 out of 240 | elapsed:    1.6s finished


Successfully saved data to: /content/drive/My Drive/Summer 2025/Python Practice/audio/npz_files/esc50_mfcc_n128_fft1024_ovlp25_seg5000ms.npz\n
Starting job for: esc50_mfcc_n128_fft2048_ovlp25_seg5000ms.npz


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done 240 out of 240 | elapsed:    1.6s finished


Successfully saved data to: /content/drive/My Drive/Summer 2025/Python Practice/audio/npz_files/esc50_mfcc_n128_fft2048_ovlp25_seg5000ms.npz\n
--- All preprocessing jobs are complete! ---
